In [1]:
import sqlite3 as sql

In [2]:
conn = sql.connect("example.db") 
cur = conn.cursor()

In [3]:
try:
    cur.executescript(open("create.sql", 'r').read())
except sql.OperationalError as err:
    print("Encountered error: " + str(err))

In [4]:
try:
    cur.executescript(open("insert_items.sql", 'r').read())
except Exception as err:
    print("Encountered error: " + str(err))

In [5]:
list_books = """
SELECT DISTINCT title FROM Books
"""
for row in cur.execute(list_books):
    print(row)

('Book1',)
('Book2',)
('Book3',)
('Book4',)


In [6]:
list_readers = """
SELECT DISTINCT name FROM Readers
"""
for row in cur.execute(list_readers):
    print(row)

('Read1',)
('Read2',)
('Read3',)


In [7]:
add_book = """
INSERT INTO Books (id, author, title, publish_year)
VALUES (NULL, "Auth4", "Book6", "1241")
"""

cur.execute(add_book)
for row in cur.execute(list_books):
    print(row)

('Book1',)
('Book2',)
('Book3',)
('Book4',)
('Book6',)


In [8]:
add_reader = """
INSERT INTO Readers (name)
VALUES ("Read7")
"""

cur.execute(add_reader)
for row in cur.execute(list_readers):
    print(row)

('Read1',)
('Read2',)
('Read3',)
('Read7',)


In [9]:
from datetime import datetime

def find_book(cursor, title):
    request = """
    SELECT DISTINCT Books.id FROM Books
        WHERE (SELECT COUNT(*) FROM Records
                WHERE Books.id == Records.book_id 
                    AND Records.returning_date IS NULL) == 0
            AND
            Books.title == ?
    """
    cursor.execute(request, (title,))
    res = cursor.fetchone()
    if res:
        return res[0]
    return None

def request_book(cursor, title, reader_id):
    book_id = find_book(cursor, title)

    if book_id:
        date = datetime.now().strftime("%d.%m.%Y")
        request = """
        INSERT INTO Records VALUES (?, ?, ?, NULL)
        """
        
        cursor.execute(request, (reader_id, book_id, date))
    else:
        raise Exception("book %s is unavailable"%(title))

try:
    request_book(cur, "Book4", 1)
except Exception as err:
    print(err)

try:
    request_book(cur, "Book6", 1)
except Exception as err:
    print(err)

for row in cur.execute("SELECT DISTINCT * FROM Records"):
    print(row)

book Book4 is unavailable
(1, 4, '18.02.2020', None)
(1, 2, '18.02.2020', '24.02.2020')
(2, 5, '04.04.2020', None)
(1, 6, '21.05.2021', None)


In [10]:
def check_request(cursor, book_id):
    cursor.execute(
        """SELECT * FROM Records WHERE book_id == %d AND returning_date is NULL""" % (book_id)
    )
    res = cursor.fetchone()
    if res:
        return True
    return False

def return_book(cursor, book_id):
    if check_request(cursor, book_id):
        date = datetime.now().strftime("%d.%m.%Y")
        request = """
        UPDATE Records SET returning_date = ?
            WHERE book_id == ? AND returning_date IS NULL
        """

        cursor.execute(request, (date, book_id))
    else:
        raise Exception("no records found")

try:
    return_book(cur, 1)
except Exception as err:
    print(err)

try:
    return_book(cur, 5)
except Exception as err:
    print(err)

for row in cur.execute("SELECT DISTINCT * FROM Records"):
    print(row)

no records found
(1, 4, '18.02.2020', None)
(1, 2, '18.02.2020', '24.02.2020')
(2, 5, '04.04.2020', '21.05.2021')
(1, 6, '21.05.2021', None)
